In [ ]:
import pandas as pd 
import itertools
import nltk
from bert_embedding import BertEmbedding
import spacy
import math
import nltk.translate.bleu_score as bleu
import nltk.translate.gleu_score as gleu

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)


try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
    
bert_E = BertEmbedding()
nlp = spacy.load('en_core_web_lg')   



def token_list(embeddings, no_sep=False):
    """
    Returns with the tokens of the embedding data from the BertEmbedding.

    Params:
        embeddings: The embedding data from BertEmbedding
        no_sep: If True, the separators are trimmed.
    Return:
        tokens: list of tokens
    """
    if no_sep:
        return embeddings[0][0][1:-1]
    return embeddings[0][0]


def sentence_embs(embeddings):
    """Return with the sentence level embeddings"""
    return embeddings[0][1][0]

def prep(sentence):
    """Return with tokens and sentence level embeddings"""
    embs = bert_E([sentence], filter_spec_tokens=False)
    tokens = token_list(embs, no_sep=True)
    se = sentence_embs(embs)
    return tokens, se


def mapfunct(x, type='exp', n=0.2):
    """
    Map 0-inf to 1-0 with some function
    
    Type:
        inverse: 1/(1+n*x)
        arctan: 1-2/pi*arctan(x)
        exp: (1/(1+n))^x
    """
    if type=='inverse':
        return 1/(1+n*x)
    if type=='arctan':
        return 1-2/math.pi*math.atan(n*x)
    if type=='exp':
        return (1/(1+n))**x
    else:
        raise(NotImplementedError("Function not implemented"))


def square_rooted(x):
    return math.sqrt(sum([a*a for a in x]))


def cosine_similarity(x,y):
    numerator = sum(a*b for a,b in zip(x,y))
    denominator = square_rooted(x)*square_rooted(y)
    return numerator/float(denominator)


df = pd.read_csv('Rules DataSet - All.csv')

bold = lambda x: f'<b>{x}</b>'

def highlight_shared(string1, string2, format_func):
    shared_toks = set(string1.split(' ')) & set(string2.split(' '))
    return ' '.join([format_func(tok) if tok in shared_toks else tok for tok in string1.split(' ') ])


listRules = []
for a, b in itertools.combinations(df[['ID','Diagram Type','Rule']].values.tolist(), 2):
    r0, e0 = prep(a[2])
    r1, e1 = prep(b[2])
    t0 = nlp(a[2])
    t1 = nlp(b[2])
    SmoothingFunction = nltk.translate.bleu_score.SmoothingFunction()
    record = {"Rule1_Id": a[0],"Rule2_Id": b[0], "Rule1_Type": a[1], "Rule2_Type": b[1], "Rule_1": a[2], "Rule_2": b[2],
              "Similar_words": highlight_shared(a[2], b[2], bold), "Cosine_similarity_score": cosine_similarity(t1.vector,t0.vector),
              "bleu2": bleu.sentence_bleu([r1], r0, smoothing_function=SmoothingFunction.method2)}
    listRules.append(record)
    
newdf = pd.DataFrame(listRules)
newdf.to_csv('cos_scores.csv', sep=',', encoding='utf-8', index=False)

In [ ]:
from IPython.core.display import HTML

HTML(df.loc[:, ['Rule', 'tokens_shared_with_other']].to_html(escape=False))

In [ ]:
newdf.sort_values(by = 'Cosine_similarity_score', ascending=False).head(50)

In [ ]:

from IPython.core.display import HTML
pd.set_option('display.max_colwidth', None)

HTML(newdf[['Rule_1','Rule_2','Similar_words','bleu2']].sort_values(by = 'bleu2',ascending=False).head(50).to_html(escape=False))

In [ ]:
newdf

Rule1_Id  Rule2_Id     Rule1_Type Rule2_Type  \
0            1         2             St         St   
1            1         3             St         St   
2            1         4             St         St   
3            1         5             St         St   
4            1         6             St         St   
...        ...       ...            ...        ...   
7016       116       118  Class and Seq      Class   
7017       116       119  Class and Seq      Class   
7018       117       118             St      Class   
7019       117       119             St      Class   
7020       118       119          Class      Class   

                                                 Rule_1  \
0     Consider two State Machines U′ and U of a clas...   
1     Consider two State Machines U′ and U of a clas...   
2     Consider two State Machines U′ and U of a clas...   
3     Consider two State Machines U′ and U of a clas...   
4     Consider two State Machines U′ and U of a clas...   
...                                                 ...   
7016  No operation can be used in a message of a seq...   
7017  No operation can be used in a message of a seq...   
7018  The joint effect of a sequence of refined oper...   
7019  The joint effect of a sequence of refined oper...   
7020   Parent Class should not have a Method with a ...   

                                                 Rule_2  \
0     Consider two State Machines U′ and U of a clas...   
1     Consider two State Machines U′ and U of a clas...   
2     Consider two State Machines U′ and U of a clas...   
3     Consider two State Machines U′ and U of a clas...   
4     Consider two State Machines U′ and U of a clas...   
...                                                 ...   
7016   Parent Class should not have a Method with a ...   
7017  A Class should not have an Attribute which as ...   
7018   Parent Class should not have a Method with a ...   
7019  A Class should not have an Attribute which as ...   
7020  A Class should not have an Attribute which as ...   

                                          Similar_words  \
0     <b>Consider</b> <b>two</b> <b>State</b> <b>Mac...   
1     <b>Consider</b> <b>two</b> <b>State</b> <b>Mac...   
2     <b>Consider</b> <b>two</b> <b>State</b> <b>Mac...   
3     <b>Consider</b> <b>two</b> <b>State</b> <b>Mac...   
4     <b>Consider</b> <b>two</b> <b>State</b> <b>Mac...   
...                                                 ...   
7016  No operation can be used in <b>a</b> message o...   
7017  No operation can be used in <b>a</b> message o...   
7018  The joint effect of <b>a</b> sequence of refin...   
7019  The joint effect of <b>a</b> sequence of refin...   
7020   Parent <b>Class</b> <b>should</b> <b>not</b> ...   

      Cosine_similarity_score     bleu2  
0                    0.984344  1.000000  
1                    0.965612  1.000000  
2                    0.985321  1.000000  
3                    0.982480  1.000000  
4                    0.979362  1.000000  
...                       ...       ...  
7016                 0.910688  0.062932  
7017                 0.896977  0.062932  
7018                 0.809314  0.069645  
7019                 0.790429  0.065867  
7020                 0.964927  0.302770  

[7021 rows x 9 columns]

In [ ]:
pd.set_option('display.max_colwidth', 50)
import numpy as np
matrix = pd.pivot_table(newdf, values='bleu2', index=['Rule2_Id'],columns=['Rule1_Id'], aggfunc=np.sum,).fillna('-')
matrix.to_csv('similarity_matrix.csv', sep=',', encoding='utf-8', index=False)


In [ ]:
mean_value = newdf[['Rule1_Id','Rule1_Type','bleu2','Cosine_similarity_score']].groupby(['Rule1_Id']).mean().sort_values(by = 'bleu2',ascending=False).round(2)
mean_value.to_csv('mean_value.csv', sep=',', encoding='utf-8', index=False)